In [1]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

import json
import gzip

from os import walk
from scipy.spatial import KDTree

# !pip install geopy
# !pip install phonenumbers
# !pip install pycountry

import geopy.distance
import phonenumbers
import pycountry



In [2]:
pd.options.display.max_columns = 100

In [3]:
path = r"../src/data/Hotel"
file_path_min3 = path + r"/Hotel_minimum3/geo_preprocessed"
file_path_top100 = path + r"/Hotel_top100/geo_preprocessed"

files_min3 = os.listdir(file_path_min3)
files_top100 = os.listdir(file_path_top100)

In [4]:
print(len(files_min3))
print(len(files_top100))

1043
53


In [5]:
LB_min3 = []

for lb in files_min3:
    with gzip.open(file_path_min3 + '/' + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LB_min3.append(lineData)
df_min3 = pd.DataFrame(LB_min3)

In [6]:
len(df_min3)

207931

In [7]:
LB_top100 = []

for lb in files_top100:
    with gzip.open(file_path_top100 + '/' + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LB_top100.append(lineData)
df_top100 = pd.DataFrame(LB_top100)

In [8]:
len(df_top100)

316809

## Concatenate Dataframes

In [9]:
df_all = pd.concat([df_min3, df_top100], axis = 0, ignore_index = True)

In [10]:
len(df_all)

524740

In [11]:
df_all.tail()

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,...,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee
524735,5185,Supalai Resort & Spa Phuket,"Moo 6 Ao Po Lang Daeng Road Paklok Thala,",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/thailand/p...,NaN,NaN,NaN,NaN,NaN,+66 763 02302,NaN,"{'reviewcount': '140', 'ratingvalue': '3.5E0'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524736,5186,Oba Star Hotel,"Oba Mahallesi, Obagöl Mevkii, Mesut Cd., 07640...",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ant...,NaN,NaN,NaN,NaN,NaN,0090 242 514 32 83,NaN,"{'ratingvalue': '4.5E0', 'reviewcount': '1251'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524737,5187,Hotel Askoc,"Istasyon Arka Sokak No 15Sirkeci 34420, Istanbul",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ist...,NaN,NaN,NaN,NaN,NaN,902125118089,NaN,"{'reviewcount': '148', 'ratingvalue': '3.5E0'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524738,5188,Hotel Xapala,"Carrer d'Amílcar, 5, 07600 El Arenal, Illes Ba...",NaN,NaN,{'ratingvalue': '0'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/spain/bale...,NaN,NaN,NaN,NaN,NaN,None,NaN,"{'ratingvalue': '4.0E0', 'reviewcount': '82'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524739,5189,Hotel Best Western Amber,"Muhsine Hatun Mah.Ciftegelinler Cad.No:3/7, Is...",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ist...,NaN,NaN,NaN,NaN,NaN,+90 212 518 48 01,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Keep where dataframe has non-zero telephone numbers AND non-zero country codes

In [12]:
df_clean = df_all[df_all["addresscountry"].notna()]
df_clean = df_clean[df_clean["telephone"].notna()]
len(df_clean)

158543

## Format longitudes AND latitudes

In [13]:
lon = "longitude"
lat = "latitude"

In [14]:
# Remove entries that are not numbers or cannot be convertred to one number (list etc.)
longitudes = df_clean[lon].to_numpy()
latitudes = df_clean[lat].to_numpy()
deleteList = []
i = 0

for value in longitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1

i = 0
for value in latitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1    

df_clean.drop(df_clean.index[deleteList], axis = 0, inplace = True)        

### Format longitude and latitude

In [15]:
longArray = df_clean[lon].to_numpy().astype(str)
longArray = np.char.replace(longArray, ',', '.')
longArray = np.char.replace(longArray, '--', '-')
df_clean[lon] = longArray
df_clean[lon] = pd.to_numeric(df_clean[lon], errors='coerce')

latArray = df_clean[lat].to_numpy().astype(str)
latArray = np.char.replace(latArray, ',', '.')
latArray = np.char.replace(latArray, '--', '-')
df_clean[lat] = latArray
df_clean[lat] = pd.to_numeric(df_clean[lat], errors='coerce')

In [16]:
# Remove the entries that were set to NaN because of other errors
df_clean = df_clean[df_clean["longitude"].notna()]
df_clean = df_clean[df_clean["latitude"].notna()]
# Make sure to only include valid longitudes and latitudes
df_clean = df_clean.loc[(df_clean[lat] >= -90) & (df_clean[lat] <= 90)]
df_clean = df_clean.loc[(df_clean[lon] >= -180) & (df_clean[lon] <= 80)]

In [17]:
len(df_clean)

67570

## Second preprocessing step

### Remove non-digits from telephone numbers

In [83]:
def remove_non_digits(string):
    cleaned = re.sub('[^0-9]','', string)
    return cleaned

In [84]:
df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)

### Extract country codes to ISO-2 format using ``pycountry``

In [85]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

countries


{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [86]:
# fuction to modify the country dictionary in uppercase
def modify_dic(d):
    for key in list(d.keys()):
        new_key = key.upper()
        d[new_key] = d[key]
        print(new_key)
    return d

In [87]:
countries_upper = modify_dic(countries)

ARUBA
AFGHANISTAN
ANGOLA
ANGUILLA
ÅLAND ISLANDS
ALBANIA
ANDORRA
UNITED ARAB EMIRATES
ARGENTINA
ARMENIA
AMERICAN SAMOA
ANTARCTICA
FRENCH SOUTHERN TERRITORIES
ANTIGUA AND BARBUDA
AUSTRALIA
AUSTRIA
AZERBAIJAN
BURUNDI
BELGIUM
BENIN
BONAIRE, SINT EUSTATIUS AND SABA
BURKINA FASO
BANGLADESH
BULGARIA
BAHRAIN
BAHAMAS
BOSNIA AND HERZEGOVINA
SAINT BARTHÉLEMY
BELARUS
BELIZE
BERMUDA
BOLIVIA, PLURINATIONAL STATE OF
BRAZIL
BARBADOS
BRUNEI DARUSSALAM
BHUTAN
BOUVET ISLAND
BOTSWANA
CENTRAL AFRICAN REPUBLIC
CANADA
COCOS (KEELING) ISLANDS
SWITZERLAND
CHILE
CHINA
CÔTE D'IVOIRE
CAMEROON
CONGO, THE DEMOCRATIC REPUBLIC OF THE
CONGO
COOK ISLANDS
COLOMBIA
COMOROS
CABO VERDE
COSTA RICA
CUBA
CURAÇAO
CHRISTMAS ISLAND
CAYMAN ISLANDS
CYPRUS
CZECHIA
GERMANY
DJIBOUTI
DOMINICA
DENMARK
DOMINICAN REPUBLIC
ALGERIA
ECUADOR
EGYPT
ERITREA
WESTERN SAHARA
SPAIN
ESTONIA
ETHIOPIA
FINLAND
FIJI
FALKLAND ISLANDS (MALVINAS)
FRANCE
FAROE ISLANDS
MICRONESIA, FEDERATED STATES OF
GABON
UNITED KINGDOM
GEORGIA
GUERNSEY
GHANA
GIBRALTAR
GUI

In [88]:
countries_upper

{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [89]:
#uppercase the df_column 
df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()

In [90]:
# Replace known countrires with ISO-2 format country code
for key, value in countries_upper.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

<ipython-input-90-d03c22551531>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)


In [91]:
df_clean["addresscountry"].unique()

array(['US', 'CA', 'MX', 'PR', 'JM', 'AD', 'GB', 'FR', 'DK', 'IE', 'PT',
       'GG', 'IN', 'ZA', 'IT', 'US OF AMERICA', 'AE', 'HU', 'JE', 'NO',
       'TH', 'SA', 'ES', 'MV', 'QA', 'GR', 'MT', 'DE', 'LU', 'NEIA',
       'CZECH REPUBLIC', 'NL', 'IM', 'SE', 'PL', 'CH', 'BH', 'AF',
       'WALES', 'ESPAÑA', 'USA', 'UK', 'MA', 'SCT', 'CY', 'ENG', 'WAL',
       'OM', 'KW', 'EN', 'BG', 'PUTEAUX', 'BUSSY SAINT GEORGES',
       'SAINT OUEN L AUMONE', 'AUBERVILLIERS', 'PARIS', 'RE', 'TR', 'RO',
       'UA', 'SI', 'BA', 'HR', 'RU', 'AT', 'CZ', 'MK', 'LV', 'EE', 'AL',
       'LT', 'CS', 'BY', 'AU', '162', 'IL', 'LB', 'ENGLAND', 'ROMIA',
       'GA', 'ME', 'SN', 'BR', 'ET', 'RUSSIA', 'TN', 'CL', 'PE', 'THE NL',
       'EG', 'CU', 'UG', 'SCOTLAND', 'CO', 'DJ', 'AR', 'BZ', 'KE', 'FI',
       'MG', 'BE', 'TANZANIA', 'NORTHERN IE', 'RW', 'SM', 'JP',
       'THIS LOCATION OPEN FOR LUNCH ONLY', 'GP', 'MQ', 'GBR', 'FRA',
       'POL', 'SWE', 'HRV', 'DEU', 'PRT', 'ITA', 'KY', 'SK', 'GE', 'RS',
       'TC

## Manually normalize countries which do not exist in country package

In [92]:
df_clean["addresscountry"].value_counts().head(30)

US        51382
FR        14603
UK        13907
BR        13759
IN         6507
AE         4206
GB         4050
DK         2733
ES         2498
CA         2452
UAE        1878
ITALIA     1729
MX         1612
FRA        1362
USA        1108
IT         1050
RU          983
DEU         965
TR          799
AR          658
DE          649
QA          620
BH          616
CO          483
BE          439
IE          420
NO          398
JO          323
PL          317
ZA          305
Name: addresscountry, dtype: int64

In [93]:
country_dictionary = {
                      "UNITED STATES": "US",
                      "USA":"US",
                      "UNITED KINGDOM": "GB",
                      "UK": "GB",
                      "CANADA": "CA",
                      "AUSTRALIA": "AU",
                      "UNITED ARAB EMIRATES":"AE",
                      "INDIA" : "IN",
                      "NEW ZEALAND": "NZ",
                      "SVERIGE" : "SE",
                      "DEUTSCHLAND": "DE",
                      "RUSSIA": "RU",
                      "ITALIA": "IT",
                      "IRAN": "IR",
                      ", IN" : "IN",
                      "ENGLAND": "GB"
                    }
for key, value in country_dictionary.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

## In this manual step we save about 43.000 extra datapoints

## Remove non-covered countries

### There are still some uncovered cases left which have to be removed

In [94]:
df_clean.reset_index(inplace=True)

In [95]:
liste = []
for i, row in enumerate(df_clean["addresscountry"]):
    if len(row) > 2:
        liste.append(i)
        
df_clean = df_clean.drop(liste)        

In [96]:
df_clean["addresscountry"].unique()

array(['US', 'CA', 'MX', 'PR', 'JM', 'AD', 'GB', 'FR', 'DK', 'IE', 'PT',
       'GG', 'IN', 'ZA', 'IT', 'AE', 'HU', 'JE', 'NO', 'TH', 'SA', 'ES',
       'MV', 'QA', 'GR', 'MT', 'DE', 'LU', 'NL', 'IM', 'SE', 'PL', 'CH',
       'BH', 'AF', 'MA', 'CY', 'OM', 'KW', 'EN', 'BG', 'RE', 'TR', 'RO',
       'UA', 'SI', 'BA', 'HR', 'RU', 'AT', 'CZ', 'MK', 'LV', 'EE', 'AL',
       'LT', 'CS', 'BY', 'AU', 'IL', 'LB', 'GA', 'ME', 'SN', 'BR', 'ET',
       'TN', 'CL', 'PE', 'EG', 'CU', 'UG', 'CO', 'DJ', 'AR', 'BZ', 'KE',
       'FI', 'MG', 'BE', 'RW', 'SM', 'JP', 'GP', 'MQ', 'KY', 'SK', 'GE',
       'RS', 'TC', 'CR', 'CW', 'IS', 'JO', 'GT', 'DM', 'UY', 'AW', 'LC',
       'PK', 'MD', 'PH', 'PF', 'VU', 'MC', 'BO', 'EC', 'DO', 'LK', 'LI',
       'DZ', 'PY', 'NI', 'HN', 'BL', 'BS'], dtype=object)

## Drop empty phonenumbers and too lenghty phone numbers

In [97]:
df_clean = df_clean[df_clean["telephone_"] != "" ]

In [98]:
liste = []
df_clean.reset_index(inplace=True)
for row_index in df_clean.index:
    if len(df_clean.iloc[row_index]["telephone_"])>18:
        liste.append(row_index)

In [99]:
df_clean.drop(labels = liste, inplace = True)

In [100]:
df_clean = df_clean.drop(columns = ["level_0","index"])
df_clean.tail()

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,...,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_
131895,9163,FOOW,{'streetaddress': '34 Goldenrod Circle Santa R...,"{'reviewcount': '714', 'ratingvalue': '4.4E0'}",NaN,NaN,NaN,NaN,https://www.opentable.com/r/foow-santa-rosa-beach,FL,Santa Rosa Beach,"34 Goldenrod Circle Santa Rosa Beach, FL 32459",32459,(850) 534-5050,"{'longitude': '-86.1548260', 'latitude': '30.3...",Please note: Fish Out of Water no longer accep...,NaN,US,-86.154826,30.356375,"Contemporary American, Seafood, Southern",$31 to $50,NaN,"[{'name': 'Brunch', 'description': '8 AM – 2 P...",{'openinghours': 'Brunch <br />Daily 8:00 am–2...,NaN,NaN,NaN,Brunch <br />Daily 8:00 am–2:00 pm <br />Happy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'reviewbody': 'The food and the service we g...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8505345050
131896,9164,Sushi By Bou at the Former Versace Mansion,"{'addressregion': 'FL', 'name': 'Sushi By Bou ...","{'ratingvalue': '4.8E0', 'reviewcount': '80'}",NaN,NaN,NaN,NaN,https://www.opentable.com/r/sushi-by-bou-versa...,FL,Miami Beach,"1116 Ocean Dr Miami Beach, FL 33139-4609",33139-4609,305922919598398,"{'latitude': '25.7818950', 'longitude': '-80.1...",At the iconic former Versace Mansion in South ...,NaN,US,-80.130777,25.781895,"Sushi, Asian, Japanese",$50 and over,NaN,None,"{'pricerange': '$50 and over', 'telephone': '3...",NaN,NaN,NaN,Dinner <br />Tue–Sun 5:00 pm–10:00 pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'datepublished': '2020-06-09T18:04:45Z', 're...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,305922919598398
131897,9165,Preserved Restaurant,{'streetaddress': '102 Bridge St Florida St. A...,"{'reviewcount': '2865', 'ratingvalue': '4.8E0'}",NaN,NaN,NaN,NaN,https://www.opentable.com/preserved-restaurant...,FL,St. Augustine,"102 Bridge St Florida St. Augustine, FL 32084",32084,(904) 679-4940,"{'latitude': '29.8887650', 'longitude': '-81.3...",Preserved Restaurant is an ingredient-driven r...,NaN,US,-81.316076,29.888765,"Southern, French",$30 and under,NaN,None,"{'telephone': '(904) 679-4940', 'name': 'Prese...",NaN,NaN,NaN,Supper <br />Wed–Sun 3:00 pm–8:00 pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'reviewbody': 'Wonderful food and staff.', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9046794940
131898,9169,Coqbull,"{'addresscountry': 'IE', 'addresslocality': 'C...","{'reviewcount

In [101]:
len(df_clean)

131328

## Define normalizer for telephone package phonenumbers

In [102]:
def normalizer(entity):
    number = entity["telephone_"]
    address_country = entity["addresscountry"]
    phone_number = phonenumbers.parse(number, address_country)
    return phone_number

## Finally normalizing phone numbers in E.164 format

### Ignore those which can not be identified and replace as ``unknown``

In [103]:
df_clean.reset_index(inplace=True)
phone_objects =[]
#index = []
for row_index in df_clean.index:
    try:
        phone_object =  normalizer(df_clean.iloc[row_index])
        #index.append(row_index)
        phone_objects.append(phone_object)
    except:
        phone_objects.append("unknown")
    
    

In [104]:
len(phone_objects)

131328

In [105]:
df_clean["phone_object"] = pd.Series(phone_objects)

In [106]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,...,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object
0,0,West End Tavern,"{'streetaddress': '926 Pearl St.', 'addressreg...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Boulder,926 Pearl St.,80302,(303) 444-3535,"{'longitude': '-105.283247', 'latitude': '40.0...",None,{'closes': 'false'},US,-105.283247,40.016997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3034443535,Country Code: 1 National Number: 3034443535
1,1,A Piece of Cake,"{'postalcode': '30305', 'addresslocality': 'At...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,GA,Atlanta,3215 Roswell Rd NE,30305,800.922.5390,"{'latitude': '33.84289', 'longitude': '-84.380...",Piece of Cake has been baking freshly made cak...,{'closes': 'false'},US,-84.380739,33.842890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8009225390,Country Code: 1 National Number: 8009225390
2,2,Revel,"{'addressregion': 'WA', 'postalcode': '98103',...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,WA,Seattle,403 N 36th Street,98103,(206) 547-2040,"{'longitude': '-122.3562456', 'latitude': '47....",Facebook: Revel Twitter: @revelseattle,{'closes': 'false'},US,-122.356246,47.651964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2065472040,Country Code: 1 National Number: 2065472040
3,4,SFT Bistro,{'streetaddress': '1920 Tigertail Blvd Buildin...,NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,FL,Dania Beach,1920 Tigertail Blvd Building 12,33004,(954) 234-2327,"{'latitude': '26.052338', 'longitude': '-80.16...","Head to Dania Beach, Fla., where the SFT Bistr...",{'closes': 'false'},US,-80.163356,26.052338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9542342327,Country Code: 1 National Number: 9542342327
4,5,Pink Box Doughnuts,"{'streetaddress': '7531 W. Lake Mead Blvd', 'a...",NaN,NaN,NaN,NaN,NaN,htt

In [107]:
unknown_rows = df_clean[df_clean["phone_object"] == "unknown"].index

In [108]:
df_clean = df_clean.drop(unknown_rows)

In [109]:
len(df_clean)

131214

## Check whether phonenumbers are valid

In [110]:
df_valid_numbers = df_clean[df_clean["phone_object"].apply(phonenumbers.is_valid_number)]

In [111]:
len(df_valid_numbers)

116051

## Next step: Format every telephone number into unique E.164 format

In [112]:
#phonenumbers.format_number(df_valid_numbers["phone_object"][0], phonenumbers.PhoneNumberFormat.E164)

In [113]:
df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda objects: phonenumbers.format_number(objects, phonenumbers.PhoneNumberFormat.E164))

<ipython-input-113-28dce20706cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda objects: phonenumbers.format_number(objects, phonenumbers.PhoneNumberFormat.E164))


In [114]:
len(df_valid_numbers)

116051

In [115]:
df_valid_numbers.head()

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,...,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format
0,0,West End Tavern,"{'streetaddress': '926 Pearl St.', 'addressreg...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Boulder,926 Pearl St.,80302,(303) 444-3535,"{'longitude': '-105.283247', 'latitude': '40.0...",None,{'closes': 'false'},US,-105.283247,40.016997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3034443535,Country Code: 1 National Number: 3034443535,+13034443535
1,1,A Piece of Cake,"{'postalcode': '30305', 'addresslocality': 'At...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,GA,Atlanta,3215 Roswell Rd NE,30305,800.922.5390,"{'latitude': '33.84289', 'longitude': '-84.380...",Piece of Cake has been baking freshly made cak...,{'closes': 'false'},US,-84.380739,33.842890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8009225390,Country Code: 1 National Number: 8009225390,+18009225390
2,2,Revel,"{'addressregion': 'WA', 'postalcode': '98103',...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,WA,Seattle,403 N 36th Street,98103,(206) 547-2040,"{'longitude': '-122.3562456', 'latitude': '47....",Facebook: Revel Twitter: @revelseattle,{'closes': 'false'},US,-122.356246,47.651964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2065472040,Country Code: 1 National Number: 2065472040,+12065472040
3,4,SFT Bistro,{'streetaddress': '1920 Tigertail Blvd Buildin...,NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,FL,Dania Beach,1920 Tigertail Blvd Building 12,33004,(954) 234-2327,"{'latitude': '26.052338', 'longitude': '-80.16...","Head to Dania Beach, Fla., where the SFT Bistr...",{'closes': 'false'},US,-80.163356,26.052338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9542342327,Country Code: 1 National Number: 9542342327,+19542342327
4,5,Pink Box Doughnuts,"{'streetaddress': '7531 W. Lake Mead

## After formatting phone numbers into unified format we can group by phone numbers to identify clusters

In [116]:
df_valid_numbers["E.164 format"].value_counts().sort_values().tail(100)

+33320061815       45
+33139503199       46
+442077230549      46
+33143573342       46
+33328402525       46
                 ... 
+441179372332     131
+33141099999      156
+573057341428     261
+33140467900      278
+917861004444    3577
Name: E.164 format, Length: 100, dtype: int64

## As one can see from the geo locations this is a *successful* match!

In [117]:
pd.set_option('display.max_columns', 500)
df_valid_numbers[df_valid_numbers["E.164 format"] == "+442084681087"][:5]

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format


## Note we also have many non-matches which is why we need geo-locations

## Adding the matching telephone numbers in a new column

In [118]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, r = 0):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r)
    return idx

df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
df_valid_numbers.reset_index(drop=True, inplace=True)
df_valid_numbers['indexValue'] = df_valid_numbers.index
telephoneArray = df_valid_numbers['telephoneNorm'].to_numpy().astype('int64')
fillArray = np.full(len(telephoneArray), 1)
tupleArray = np.array((telephoneArray, fillArray)).T.astype('int64')


# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
df_valid_numbers['MatchingNumbers'] = df_valid_numbers.apply(lambda row: queryTree(tree,[row['telephoneNorm'], 1]), axis=1) 

<ipython-input-118-623d5e3ddf8f>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-118-623d5e3ddf8f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-118-623d5e3ddf8f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [119]:
# filter out the values which only have one value
data = df_valid_numbers[df_valid_numbers['MatchingNumbers'].apply(lambda x: len(x) > 1)]

In [120]:
len(data)

46646

In [121]:
data.head()

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
0,0,West End Tavern,"{'streetaddress': '926 Pearl St.', 'addressreg...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Boulder,926 Pearl St.,80302,(303) 444-3535,"{'longitude': '-105.283247', 'latitude': '40.0...",None,{'closes': 'false'},US,-105.283247,40.016997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3034443535,Country Code: 1 National Number: 3034443535,+13034443535,13034443535,0,"[0, 50354]"
2,2,Revel,"{'addressregion': 'WA', 'postalcode': '98103',...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,WA,Seattle,403 N 36th Street,98103,(206) 547-2040,"{'longitude': '-122.3562456', 'latitude': '47....",Facebook: Revel Twitter: @revelseattle,{'closes': 'false'},US,-122.356246,47.651964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2065472040,Country Code: 1 National Number: 2065472040,+12065472040,12065472040,2,"[2, 49813]"
5,10,Sweet Action,"{'streetaddress': '52 Broadway', 'postalcode':...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Denver,52 Broadway,80203,303-282-4645,"{'longitude': '-104.989318', 'latitude': '39.7...",Sweet Action is playing with the boundaries of...,{'closes': 'false'},US,-104.989318,39.717578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3032824645,Country Code: 1 National Number: 3032824645,+13032824645,13032824645,5,"[28299, 5]"
8,15,Uncle Gussy's,"{'streetaddress': 'Park Ave & E 51st St.', 'ad...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,NY,New York,Park Ave & E 51st St.,10022,(929) 600-1111,"{'longitude': '-73.973424', 'latitude': '40.75...",Traditional Greek Food,{'closes': 'false'},US,-73.973424,40.757760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Additional Filtering by Geo Location

In [122]:
def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

def calcDifferenceFromRow (row):
    tmp = data
    indexValue = row['indexValue']
    indexPosition = (row[lat], row[lon])
    diffList = []
    for value in row['MatchingGeoPoints']:
        if not value in tmp.index:
            continue
        currRow = data.loc[data['indexValue'] == value]
        currIndex = currRow['indexValue'].values[0]
        if currIndex == indexValue:
            diffList.append(-1)
        else:            
            currPosition = (currRow[lat].values[0], currRow[lon].values[0])
            diffList.append(calcDifference(indexPosition, currPosition))
    return diffList


In [123]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, radius = 0.001):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r=radius)
    return idx
    #idx = tree.query(point, k=neighbours)
    #return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

data.reset_index(drop=True, inplace=True)
data['indexValue'] = data.index

# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 

# Keep those that have one or more matches withing the radius
data = data[data['MatchingGeoPoints'].apply(lambda x: len(x) > 1)]

<ipython-input-123-bb4867981968>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['indexValue'] = data.index
<ipython-input-123-bb4867981968>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1)


In [124]:
# Calculate the difference in km between those
data['Difference'] = data.apply(lambda row: calcDifferenceFromRow(row), axis=1) 

In [132]:
data.iloc[3:4]

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,MatchingGeoPoints,Difference
32,248,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,TX,Austin,4905 Airport Blvd.,78751,(512) 451-3326,"{'longitude': '-97.715239', 'latitude': '30.30...",None,{'closes': 'false'},US,-97.715239,30.30964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5124513326,Country Code: 1 National Number: 5124513326,+15124513326,15124513326,32,"[140, 47716]","[21907, 32]","[0.004145070470485397, -1]"


In [126]:
len(data)

42236

In [146]:
data.loc[data['indexValue'] == 32][['name', 'address', 'page_url', 'E.164 format', lat, lon]]

,name,address,page_url,E.164 format,latitude,longitude
32,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",https://www.cookingchanneltv.com/restaurant-gu...,+15124513326,30.30964,-97.715239


In [138]:
data.loc[data['indexValue'] == 21907]

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,MatchingGeoPoints,Difference
21907,3001,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",NaN,NaN,NaN,NaN,NaN,https://theinfatuation.com/austin/reviews/tyso...,TX,Austin,4905 Airport Blvd,78751,(512) 451-3326,"{'latitude': '3.0309656E1', 'longitude': '-9.7...",NaN,NaN,US,-97.7152,30.309656,"[Tacos, Tex-Mex]",$,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'value': 'North Loop', 'name': 'Neighborhood'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5124513326,Country Code: 1 National Number: 5124513326,+15124513326,15124513326,21907,"[140, 47716]","[21907, 32]","[-1, 0.00414507047025422]"


In [129]:
pwd

'/work-ceph/bizer-tp2021/data_integration_using_deep_learning/notebooks'

In [130]:
#data.to_json("matchingRestaurant", compression='gzip', orient='records', lines=True)